In [2]:
import numpy as np

In [3]:
data = open("C:/Users/a.ezhilarasan/Documents/AI/DeepLearning/RNN/Zayne_lyrics.txt", 'r').read()
data = data.replace('\n',' ')
data = data.replace('\\','') 

In [4]:
chars = list(set(data))
data_size, vocab_length = len(data), len(chars)
print ('Data has total', data_size, 'chars and unique chars', vocab_length)

Data has total 15279 chars and unique chars 65


In [5]:
hidden_size = 100
seq_length = 25
learning_rate = 1e-1
Wxh = np.random.randn(hidden_size,vocab_length) * 0.01
Whh = np.random.randn(hidden_size, hidden_size) * 0.01
bh = np.zeros((hidden_size, 1))
Why = np.random.randn(vocab_length, hidden_size) * 0.01
by = np.zeros((vocab_length, 1))


In [6]:
char_to_ix = {ch:i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}

In [7]:
char_to_ix['.']
np.tanh(1)
type(range(vocab_length))

range(61)

range(0, 61)

In [8]:
def lossFun(inputs, targets, hprev):
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_length,1))
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh,hs[t-1]) + bh)
        ys[t] = np.dot(Why, hs[t]) + by
        ps[t] = np.exp(ys[t])/np.sum(np.exp(ys[t]))
        loss += -np.log(ps[t][targets[t],0])
    dWxh, dWhh, dWhy, dbh, dby = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why), np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext
        dhraw = (1 - hs[t] * hs[t]) * dh
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)
    
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]
        


In [9]:
def sample(h, seed_ix, length):
    x = np.zeros((vocab_length, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(length):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_length), p=p.ravel())
        ixes.append(ix)
        x = np.zeros((vocab_length, 1))
        x[ix] = 1
    txt = ''.join(ix_to_char[ix] for ix in ixes)
    print('----------\n',txt, '\n-----------',)
    
        

In [10]:
hprev = np.zeros((hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
sample(hprev,char_to_ix['a'],200)

----------
 o:UFKtTqlEF2SHdaWMk'NrcZS CkPr[zYjww&??:C8cBa8mzPbI:cK:r'N qG:?ryccw88a-'urBP(8J]U]nSJ[WTh2"Y]YhdHDH'LLRh:LU?&"E ?ydUCp B28[RHabETfbjZ2]scmnO&lbnNkl?'w8qTpNokRo,"ScGDiTd2g,NZ'gAmT.sOy,p:q2N(AUuleyg-?S 
-----------


In [11]:
mWxh, mWhy, mWhh, mbh, mby = np.zeros_like(Wxh), np.zeros_like(Why), np.zeros_like(Whh), np.zeros_like(bh), np.zeros_like(by)
smooth_loss = -np.log(1.0/vocab_length)*seq_length
n, p = 0, 0
while n<=1000*1000:
    
    if p+seq_length+1 >= len(data) or n==0:
        hprev = np.zeros((hidden_size, 1))
        p = 0

    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+1+seq_length]]
    
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss*0.999 + loss*0.001
    
    if n % 1000 == 0:
        print('iter:',n)
        print('loss:',smooth_loss)
        sample(hprev, inputs[0],200)
    
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                  [dWxh, dWhh, dWhy, dbh, dby],
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam*dparam
        param += -learning_rate*(dparam/np.sqrt(mem + 1e-8))
    
    p += seq_length
    n += 1

iter: 0
loss: 104.35967970647476
----------
 mf' oQrfGdwCbcCU&tW?FGB)iq8jk2QiS2huPs.zF2Aso&]yJh,cMmNt8O'RFcFuNE]U.wI",S-QD?bdmNIbOdMY(zxo&tLQU8Z,e]qRA"DC.B&PY&Dj.)fGws['WnEiF)2,C&'?d(Ol[EsuMf)(AEpiPaeblHDj)cxYCPhGST f[Add)"SPufkksHvKEx.wh"uRruH? 
-----------
iter: 1000
loss: 80.57282419908067
----------
  oy Nhavn tas'I  nl:e Al whao lot pyie md, whint rooco uain hiw tharm cou Nave dhi taw, theld lis't saut atomns thif mse lonn, lan itht rie ciu'le c:d she mase rlyed fo eo 'to oulr yhiI iI tin rthe au 
-----------
iter: 2000
loss: 64.62433953773065
----------
 s bewhayd Non, syo  wpy wong it, Sot y w what wat St you whis was sot wthia wat th you you dor't you waun,, yaklm bein't wong the Nou t ut thkeng An ind,,  ou doe yoc tha lot I wayd,'t anlive sake thi 
-----------
iter: 3000
loss: 56.70021714816542
----------
 faang the my aute ae you now the Dinde tris to myea lingted on't the penegt lo jI kt An pbike my anks I pie tet wyim  lows war't mekin yley inAt to,' lnit's you, Denita Oth

iter: 32000
loss: 28.506233190199723
----------
 y wirto dong  I ingiss you wanat bee so lolke my, in whe the's shat way D ntthe wrngsthe pars It's you's uth gone Juscu gokn you doon thinn Fean the roo'bo youl I win't n?ghe wrant so love To sthimh y 
-----------
iter: 33000
loss: 28.80360322658985
----------
 Ally lore I'm so my off all your all melo tur of Iteome our fars and the pints to luth ant what you wa toof I werise that yourd bea to sthe gy, to 'Caus I't rithadtre place firingave dore git it't it  
-----------
iter: 34000
loss: 28.34530615220378
----------
 have In I whrrand o fie, ing in the I'm stot you cat so love me  I'm a lines sea can't shini:Ydup an The on the in the lowinks the thr think? fool thins the a foise shise live nakeayd all do co don't  
-----------
iter: 35000
loss: 28.437250636550324
----------
 can't sis, noghaties and shart at  mecan Heuplons bellme't jumiled You've gote it's unat bed ally Now She list liek My halla say, you've an wbuthor it Shise in this

iter: 64000
loss: 23.522008123084632
----------
 r ting ou chalking off Tat's, me loony but I what knd you wanns beve, I thin's So to lits hell yee like this Give rod wangs nike Heally, Right wim a wan the whas it gop't to ploss abough I'm a fight F 
-----------
iter: 65000
loss: 23.258896455162237
----------
 e coof  A donnits at Shing in Don't you wandy Ind sn's ind  Hey  She So hecaund bo look So Les She tain Heard am like tho torius do I wand wars bads you whene threng  Shr Feabe, wars be, I wants  I ju 
-----------
iter: 66000
loss: 23.885709629409867
----------
 She  I won't you're lee sile To sus you're it up it off I'me it tais my failins in wove Jure all in In the prisaist watce won't see I'd love the to wop ano it's your shiew She things I timer shet it w 
-----------
iter: 67000
loss: 23.425059863539055
----------
 a say that alf Weniis got offathe right shene gaye on me Wing A tike I pure ghitcreap You wea that won't of the good the all macaucking To ere, she'p now thee way

iter: 96000
loss: 21.38112236709227
----------
 I just can't wants you 'Cause I'm sas you take, tame it me it's ware chaysuis In't cam mearning is Don't Be tooks you ware whin'tp uen feard a 'Cause nelingens I whase offabatee it's do like it Low wo 
-----------
iter: 97000
loss: 20.83587256143638
----------
 ake and prone place Sois This leall pGit the conet shat you'r  I know Hod abaly oucring on a wor thin son, he them bediss ane  Ceart LMy can't taid Doof fecath ond do was off che'ri doulis doon the al 
-----------
iter: 98000
loss: 20.6020349086637
----------
  So suby I can't 't sord Jus on yours zone This 'Catcins Hery  I't So time you tay, what whed and rI wron't sha timeny  Sok the rool I just be can't stol f I'vp be right wan't some Just trie've be lir 
-----------
iter: 99000
loss: 21.264362212276282
----------
 got Feaig the don tho nowesnighing all the pall the in offacainm ting to tood alo not, like I wolling fecen the testhat you ta say whenga live of likn my coof gome ci

iter: 128000
loss: 19.12249953383841
----------
 Beredise and hom whenm sann drace Noo like this bebody thar go toof go toune stam take Myourd abbody Mrd and right, sas youf don't watteting heridy I'mes I'm start been th take in the wrong place for  
-----------
iter: 129000
loss: 19.565772179144812
----------
 e me it's canttisg hed it's oof I'm and it all al fat to go pule Ands I way take time Juse go tre plat your me in the things Os Your more a goonep los I mecka Weard about all the truth [8x]  Inns Opre 
-----------
iter: 130000
loss: 18.91125503964285
----------
 o dole, like ee clou've shit you've bide A froy dollyour whe new I wanow shendf loor my :say quces Bees Yof al ablod tees whatn., bady, it and a fany  Paknow hem Your of to it fes at in's abladin here 
-----------
iter: 131000
loss: 18.83030944862474
----------
 e  Bais es you've dongmees you wanta soolling Peay s, the tonetit I that you wanna so de I'me of makeup baby just take and had, me all clay, like them updurd She

...

iter: 864000
loss: 10.628780828302242
----------
 akn anerr I'b tame your I just my my on when sobode be don't sad the love you won't ard woriegs te and the proof p hankesa sheale the truthis buisk now Sourek up it  laink you  It's you, iI stels them 
-----------
iter: 865000
loss: 10.483562842946629
----------
 he ath and  I'm  I tinged howings time And I don Deng you things Ina, h of I, I, I, I up nee this canttysise lo's lone doulking So whe amy It's lof, rikn the blace is Sot tauthis  Al you ally baca tak 
-----------
iter: 866000
loss: 10.31721641576136
----------
 s what to lon bet it it you wannaing is  Itaking ouradise off I'm futk nond my outt Bady bo luchlf like this I way the that you But'm 'boding ike of liske words I'm eees warting Give me all of I justo 
-----------
iter: 867000
loss: 10.602815625796831
----------
 whres you wanna soin thatch,now So reep I don't like taup The plough hacey  [Ke's farssink it  [ZAYto love The hacemy fin's see, won's canttow the ,titkin Her 

iter: 896000
loss: 9.96891706310955
----------
 d the palasing in thathe thakn bed abled on  Thilin be for ghis  Because my ur pore way In the the 's you It's your for you  'Caut'bodhisg foce aghad you lise'my can't sinea losind to place to loofum  
-----------
iter: 897000
loss: 10.197200798656043
----------
  and wanna share  [Kehlani:] Don't seel think be a chind 'cause all of it off Take thee wronk, no  got that all dhe chon't wros gime I'll 'Cluce plothing for abydurd so wures hea it's our prouf kise w 
-----------
iter: 898000
loss: 10.014027230926924
----------
 wanna that is so tiwen your the shingsnenet so 'tels in You've done thot is onins baby?" Sht whrnug wheling trys  feremy, mind oll Don't time Any I'm like and ter hor thes it's it riek oof Give can't  
-----------
iter: 899000
loss: 10.21441471967044
----------
 right way In the right way, framinginsays you love Hou, love donet spear long you've hare it all the shine dhat me you've be a to lon thisvigais I'd light on And

iter: 928000
loss: 10.321817820834621
----------
 on't es o lonitsa that to stos I'visp you and and  Se ig A cautce lon't watame out like then AAn the dis It's beays kniws her well for me to my Don't sood hiveine all of you'r zineing  Noust and love  
-----------
iter: 929000
loss: 9.81980153033483
----------
 urd at all the reow Betr the shit you've been place mous Makiting This, in t takn blost be a done Andw I myee fiking Tike That thror wher ef loow I, I'm fosring in't reals to with you No know Canna sa 
-----------
iter: 930000
loss: 10.05353755783286
----------
  I whew it som boby Donmann thr the wan't see take it on fid bobles you to lose I can't see in that know ton the pith he won'to for meware Nof can the right won't see it off 'slls don't shee that ie t 
-----------
iter: 931000
loss: 10.021733493478436
----------
 t's ere abloit  Litinkingars heay the her So styo see thin right farty love Time you've gone Bussine the that you've bady And me."F wat you wanna say, no Io's st

iter: 960000
loss: 10.01037933373457
----------
 doulinetise and I got wite it ain? I habaing "Heallf fagriss Onme whaze and dals in what you want Shat you mastaing An saup you're Now see the thimy and try this ut fee the the pacher over songe you'r 
-----------
iter: 961000
loss: 10.060797602489053
----------
  you thing in to  Nop I weme all of ofra that it all do Wy, polie bedorse Githe cauch  time And all You're like the  SCaid  Tay enats Open her e[sais I'd zones lovesine this Ind Yoveep I got and about 
-----------
iter: 962000
loss: 9.696236856871124
----------
 ike you  me, moure I'm tree alw It wames I doche what you wanna say, no pame lotey futh your the my make it's is I gove in the dan't equat  I'm and rool oflyou  I'd So re, she're light it all up up th 
-----------
iter: 963000
loss: 10.049918191511665
----------
 l sis out war you're lloke oration y crost the things you, jurd liek my yotr Your cout all that it your filly bahangs I'ver wostong son'boull ghat like you take

iter: 992000
loss: 9.953612991153454
----------
 As I'm ut you wint, watce I, I just to give to h,te it's paradise things I leal the place for was I  Clone I've baceuscenaist was you all ecause off I just want to wutce lyee trosse right ollond Yowhe 
-----------
iter: 993000
loss: 9.899011324308846
----------
  now abecane ol mifickere I'me of 'Cause pame it off LAve light cau's sake of feart alwain time And pait think you the weatt an Fowasas you take it off Take o foole when your way whaut se ain Don't to 
-----------
iter: 994000
loss: 10.200397349373251
----------
 sa nee war zont You're light way Don'thilgouse shotimo what in the paradief ond blay, what you wanna So say what you wanna say, what you won't way So pamaby that she say, bed the seartes you wit holle 
-----------
iter: 995000
loss: 9.917069320836445
----------
 lineaks you won's  Tais I'd to files You'r  I wolis I wims  Sha tyou to peve me like this  Itove be a ceepes I'm finking ploof I'm uperers Think dither  Toing, i